In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv")
df = df[['LaunchSite', 'PayloadMass', 'Outcome', 'Class']]

sites_list = list(df['LaunchSite'].unique())
sites_list.append('All')

plmass_list = df['PayloadMass'].to_list()
plmass_list.sort()


app = dash.Dash(__name__)

app.layout = html.Div(children=[
                                #title
                                html.H1('Dashboard SpaceX Success Outcome',
                                        style={'textAlign':'center', 'font-size':40, 'margin-bottom':20}),

                                html.Div([
                                    html.Div([

                                        html.H2('Choose a launch site:', style={'font-size':24}),
                                        #DropDown with Launch sites
                                        html.Div(dcc.Dropdown(id='input-site',
                                                                options=[{'label':i, 'value':i} for i in sites_list],
                                                                placeholder='Select a site',
                                                                value='All')
                                        ),

                                        #Pie plot
                                        html.Div(dcc.Graph(id='pie_plot'))
                                    ], style={'width':'50%'}),

                                    html.Div([

                                        html.H2('Choose a payload value with the following slider:', style={'font-size':24}),
                                        #Slider with payload mass
                                        html.Div(dcc.Slider(id='payload-slider',
                                                            min=1000,
                                                            max=16000,
                                                            step=500,
                                                            value=1000)
                                        ),

                                        html.Div(id='updatemode-output-container', style={'margin-top': 20}),

                                        html.Div(dcc.Graph(id='scatter_plot'))
                                    ], style={'width':'50%'})
                                ], style={'display': 'flex'})
                            ], style={'margin':'100px'}
)


@app.callback(Output(component_id='pie_plot', component_property='figure'),
              Input(component_id='input-site', component_property='value')) 

#Callback function for pie chart
def pie_plot_function(site):

    if site == 'All':
        data = df['Class'].value_counts().to_frame()
        title = 'Count of success launches for all sites'
    else:
        data = df.query('LaunchSite == @site')['Class'].value_counts().to_frame()
        title = f'Count of success launches for {site} site'

    data.rename(index={0:'Fail', 1:'Success'}, inplace=True)
    pie_plot = px.pie(names=data.index, values=data['Class'], title=title)

    return pie_plot



@app.callback(Output(component_id='scatter_plot', component_property='figure'),
              Input(component_id='payload-slider', component_property='value')) 

#Callback function for scatter chart
def scatter_plot_function(payload):
    # payload = 3000
    array = np.zeros(len(df['Class'])) + payload
    scatter_plot = px.scatter(df, x=df['Outcome'], y=array, 
                                color=df['Class'].apply(lambda x: 'Fail' if x==0 else 'Success'),
                                title=f'Scatter with payload = {payload}')
    return scatter_plot


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)